In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib
import h5py
import sys
import scipy.io as sio
import skimage.io

sys.path.append("../Visualization/")

from optparse import OptionParser
from Preprocess_Utilities import *
from Visualization import *

/home/admin-u6142160/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
parser = OptionParser()

parser.add_option("-m", "--image", dest="img_path")
parser.add_option("-r", "--road", dest="road_dir_path")
parser.add_option("-s", "--save_dir", dest="save_dir_path")
parser.add_option("--step", type="int", dest="step")
parser.add_option("-d", "--divide", action="store_true", dest="divide")
parser.add_option("-n", "--name", dest="name")

(options, args) = parser.parse_args()

img_path = options.data_path
road_dir_path = options.road_dir_path
save_dir_path = options.save_dir_path
step = options.step
divide = options.divide

if not img_path:
    img_path = '/localdata/u6142160/Data/090085/All_Data/090085_20170531.h5'
if not road_dir_path:
    road_dir_path = '/localdata/u6142160/Data/090085/Road_Data/'
if not save_dir_path:
    print('no save_dir_path')
    sys.exit()
if not step:
    print('no step specified')
    sys.exit()
if not name:
    name = "patch_set.h5"

save_dir_path = save_dir_path.strip('/') + '/'
print("will be saved into ", save_dir_path, name)

In [2]:
img_path = '../Data/090085/All_Data/090085_20170531.h5'
road_dir_path = '../Data/090085/Road_Data/'

In [3]:
# create road mask paths list
road_type = np.array(["motorway", "trunk", "primary", "secondary", "tertiary", "unclassified", "track", # 0-6
                      "residential", "service", "road", "living_street", # 7-10
                      "all_roads"]) # 11 
#                       "motor_trunk_pri_sec_tert_uncl_track", "motor_trunk_pri_sec_tert_uncl"]) # 12-13

road_mask_path = np.char.add(road_dir_path, 
                             np.char.add(np.char.add(road_type, '/'), np.char.add(road_type, '.tif')))

In [4]:
# read in raw image
raw_image = np.array(h5py.File(img_path)['scene'])

# read in road mask
print('road mask loaded in ...')
road_img_list = []
for cur_path in road_mask_path:
    print(cur_path)
    road_img_list.append(skimage.io.imread(cur_path))

road_img_list = np.array(road_img_list)

# assert 0-1 coding
assert (np.logical_or(road_img_list == 1, road_img_list == 0)).all()

road mask loaded in ...
../Data/090085/Road_Data/motorway/motorway.tif
../Data/090085/Road_Data/trunk/trunk.tif
../Data/090085/Road_Data/primary/primary.tif
../Data/090085/Road_Data/secondary/secondary.tif
../Data/090085/Road_Data/tertiary/tertiary.tif
../Data/090085/Road_Data/unclassified/unclassified.tif
../Data/090085/Road_Data/track/track.tif
../Data/090085/Road_Data/residential/residential.tif
../Data/090085/Road_Data/service/service.tif
../Data/090085/Road_Data/road/road.tif
../Data/090085/Road_Data/living_street/living_street.tif
../Data/090085/Road_Data/all_roads/all_roads.tif


In [5]:
# combine the road mask
print("Used masks ... ")
combined_road_mask = 0
for i in [0,1,2,3,4,5,6]:
    print(road_mask_path[i])
    combined_road_mask += road_img_list[i]
print(combined_road_mask.shape, (combined_road_mask > 1).any())

combined_road_mask[np.where(combined_road_mask > 1)] = 1
assert (np.logical_or(combined_road_mask == 1, combined_road_mask == 0)).all()

Used masks ... 
../Data/090085/Road_Data/motorway/motorway.tif
../Data/090085/Road_Data/trunk/trunk.tif
../Data/090085/Road_Data/primary/primary.tif
../Data/090085/Road_Data/secondary/secondary.tif
../Data/090085/Road_Data/tertiary/tertiary.tif
../Data/090085/Road_Data/unclassified/unclassified.tif
../Data/090085/Road_Data/track/track.tif
(7961, 8091) True


In [6]:
# save the combined road mask
save_dir_path = "../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/"
skimage.io.imsave(save_dir_path + 'road_mask.tif', combined_road_mask)

/home/admin-u6142160/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/road_mask.tif is a low contrast image
  warn('%s is a low contrast image' % fname)


In [5]:
# reload combined road mask
combined_road_mask = skimage.io.imread("../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/road_mask.tif")

split image in a balanced way

In [25]:
# pass in the road mask, split on the spcified axis (default to row => up-down split)
def searching_split(raw_image, road_mask, axis=0):

    def get_index_range(i, axis=0):
        if axis: # split from col            
            if i > 5456:
                a = (i-1652)/3.686795
                b = (i-7766)/-0.2988061
            elif i >1752:
                a = (i-1652)/3.686795
                b = (i+23831)/3.787836
            else:
                a = (i-1759)/-0.2625974
                b = (i+23831)/3.787836
            
        else: # split from row
            if i > 6318:
                a = 3.787836*i-23831
                b = -0.2988061*i+7766
            elif i >1534:
                a = -0.2625974*i+1759
                b = -0.2988061*i+7766
            else:
                a = -0.2625974*i+1759
                b = 3.686795*i+1652
        return int(a), int(b)
    
    road_cnt = np.zeros(road_mask.shape[axis])
    print(road_cnt.size)
    for i in range(road_cnt.size):
        start, end = get_index_range(i, axis)
        
        if end-start > 0:
            if axis == 0 and (raw_image[0][i] != -9999).any():
                road_cnt[i] = road_mask[i, start:end+1].sum()
            elif axis == 1 and (raw_image[0][:, i] != -9999).any():
                road_cnt[i] = road_mask[start:end+1,i].sum()

    min_diff = road_cnt.sum() + 1
    idx_list = []
    diff_list = []
    
    for i in range(1,road_cnt.size):
        cur_diff = abs((road_cnt[:i].sum() / i) - (road_cnt[i:].sum() / (road_cnt.size-i)))
        
        if cur_diff < min_diff:
            min_diff = cur_diff
            
            idx_list.append(i)
            diff_list.append(min_diff)
    
    diff_list = np.array(diff_list)
    idx_list = np.array(idx_list)
    
    return idx_list[np.where(diff_list == min_diff)[0]], min_diff

In [20]:
print(searching_split(raw_image, combined_road_mask, 0))
print()
print(searching_split(raw_image, combined_road_mask, 1))

(array([1217]), 0.03884654523549358)

(array([2971]), 0.01526001346348238)


In [28]:
print(searching_split(raw_image[:,:1217,:], combined_road_mask[:1217,:], 0))
print()
print(searching_split(raw_image[:,:1217,:], combined_road_mask[:1217,:], 1))

1217
(array([1]), 58.157072368421055)

8091
(array([2923]), 0.008933101302893931)


In [6]:
show_image_against_road(raw_image, combined_road_mask, size=-1, figsize=(150,150),
                        show_plot=False, close_plot=False)
# split by row
plt.axhline(1170) # selected

# split by col
plt.axvline(3171) # selected
plt.axvline(2923)
plt.savefig('./Result/data_split.png')
plt.close()

create top-left coordinate

In [17]:
# # split by row
# plt.axhline(1170) # selected

# # split by col
# plt.axvline(3171) # selected

divide = True
step = 8
save_dir_path = "../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/"
name = "posneg_topleft_coord_split_8"

create_set_with_name(raw_image=raw_image[:, 1170:, :], combined_road_mask=combined_road_mask[1170:, :],
                     divide = divide, step = step, save_dir_path = save_dir_path,
                     name = name+"_train")

pos coordinates' shape= (320813, 2)
neg coordinates' shape= (22784183, 2)
saved into  ../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/posneg_topleft_coord_split_8_train


In [18]:
create_set_with_name(raw_image=raw_image[:, :1170, :3171], combined_road_mask=combined_road_mask[:1170,:3171],
                     divide = divide, step = step, save_dir_path = save_dir_path,
                     name = name+"_test")
create_set_with_name(raw_image=raw_image[:, :1170, 3171:], combined_road_mask=combined_road_mask[:1170, 3171:],
                     divide = divide, step = step, save_dir_path = save_dir_path,
                     name = name+"_cv")

pos coordinates' shape= (30350, 2)
neg coordinates' shape= (1305067, 2)
saved into  ../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/posneg_topleft_coord_split_8_test
pos coordinates' shape= (21651, 2)
neg coordinates' shape= (1012131, 2)
saved into  ../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/posneg_topleft_coord_split_8_cv


In [ ]:
# # plot out some patches
# # [0-21450]
# index = 1000

# has_road = road_existence[index]
# mask = road_patch[index]
# patch = np.array([image_patch[index][3].T, image_patch[index][2].T, image_patch[index][1].T]).T

# print(has_road, patch.max(), patch.min(), patch.shape, mask.shape)
# patch = patch * 0.0001 * 5

# # plot raw img
# plt.imshow(patch)
# plt.show()
# plt.clf()

# # plot road mask
# plt.imshow(mask, cmap=plt.get_cmap('gray'))
# plt.show()
# plt.clf()